In [1]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Script to run and export ISSM inversion
# Mikayla Pascual 07/13/22
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

#import 
from MatlabFuncs import *
from model import *
from triangle import *
from bamg import bamg
from savevars import *
import plotdoc
from loadmodel import *
from os.path import exists

from scipy.io import loadmat
from m1qn3inversion import *
import numpy as np
from ContourToNodes import *
from solve import *

In [2]:
# Load some mat files to use later
vel_mat = loadmat('Cheat_matfiles/vel.mat')
hmaxVertices_mat = loadmat('Cheat_matfiles/hmaxVertices.mat')
param_mat = loadmat('Cheat_matfiles/param.mat')

velx = vel_mat['velx']
vely = vel_mat['vely']
vel = vel_mat['vel']

hmaxVertices = hmaxVertices_mat['hmaxVertices']


In [3]:
# Setup
region = 'SAtoES';
start_year = 1985;

# Mesh sizing
triangleresolution = 1000;
hmin = 300;
hmax = 10000;

# Mesh
md = model()
md = triangle(md,'./Exp/' +region+ '.exp',triangleresolution) # set up mesh

md = bamg(md,'hmin',hmin,'hmax',hmax,'field',vel,'err',2,'hmaxVertices',hmaxVertices);

#savevars('./Models/' +region+ '_mesh', 'md', md)

# 

In [5]:
# Load parameters
md.geometry.bed = param_mat['geometry_bed']
md.geometry.surface = param_mat['geometry_surface']
md.geometry.base = param_mat['geometry_base']
md.mask.ice_levelset = param_mat['mask_ice_levelset']
md.mask.groundedice_levelset = param_mat['mask_groundedice_levelset']
md.inversion.vx_obs = param_mat['inversion_vx_obs']
md.inversion.vy_obs = param_mat['inversion_vy_obs']
md.inversion.vel_obs = param_mat['inversion_vel_obs']
md.initialization.vx = param_mat['initialization_vx']
md.initialization.vy = param_mat['initialization_vy']
md.initialization.vz = param_mat['initialization_vz']
md.initialization.vel = param_mat['initialization_vel']
md.friction.coefficient = param_mat['friction_coefficient']
md.materials.rheology_n = param_mat['rheology_n']
md.materials.rheology_B = param_mat['rheology_B']
md.basalforcings.groundedice_melting_rate = param_mat['basalforcings_groundedice_melting_rate']
md.basalforcings.floatingice_melting_rate = param_mat['basalforcings_floatingice_melting_rate']
md.smb.mass_balance = param_mat['smb_mass_balance']
md.basalforcings.geothermalflux = param_mat['basalforcings_geothermalflux']
md.stressbalance.spcvx = param_mat['stressbalance_spcvx']
md.stressbalance.spcvy = param_mat['stressbalance_spcvy']
md.stressbalance.spcvz = param_mat['stressbalance_spcvz']
md.stressbalance.referential = param_mat['stressbalance_referential']
md.stressbalance.loadingforce = param_mat['stressbalance_loadingforce']
md.masstransport.spcthickness = param_mat['masstransport_spcthickness']

# Set bed = base for grounded ice
md.geometry.base = md.geometry.bed
md.geometry.thickness = md.geometry.surface - md.geometry.base

md.friction.q = np.ones(md.mesh.numberofelements)
md.friction.p = np.ones(md.mesh.numberofelements)

md.flowequation.isSSA = 1
md.flowequation.vertex_equation = 2 * np.ones(md.mesh.numberofvertices)
md.flowequation.element_equation = 2 * np.ones(md.mesh.numberofelements)
md.flowequation.borderSSA = np.ones(md.mesh.numberofvertices)
md.flowequation.borderHO = np.ones(md.mesh.numberofvertices)
md.flowequation.borderFS = np.ones(md.mesh.numberofvertices)

md.mask.ocean_levelset = -md.mask.ice_levelset

from generic import generic
md.cluster = generic() #telling where to run issm

md.miscellaneous.name = 'SAtoES'

In [6]:
import importlib
import generic_daskhub
importlib.reload(generic_daskhub)
from generic_daskhub import generic_daskhub

from socket import gethostname
md.cluster = generic_daskhub('name', gethostname(), 'np', 2)

md.settings.waitonlock = 9999 #np.inf
md.cluster.interactive = 0

from solve import solve
md = solve(md, 'sb')


checking model consistency
marshalling file SAtoES.bin
uploading input file and queueing script
launching solution sequence on remote cluster
waiting for /efs/issm/issm/trunk-jpl/execution/SAtoES-09-02-2022-19-03-59-1742/SAtoES.lock hold on... (Ctrl+C to exit)
loading results from cluster
cp /efs/issm/issm/trunk-jpl/execution/SAtoES-09-02-2022-19-03-59-1742/SAtoES.outlog ./
cp /efs/issm/issm/trunk-jpl/execution/SAtoES-09-02-2022-19-03-59-1742/SAtoES.errlog ./
cp /efs/issm/issm/trunk-jpl/execution/SAtoES-09-02-2022-19-03-59-1742/SAtoES.outbin ./


In [14]:
md.results.StressbalanceSolution.outlog

['',
 'Ice Sheet System Model (ISSM) version  4.19',
 '(website: http://issm.jpl.nasa.gov contact: issm@jpl.nasa.gov)',
 '',
 'call computational core:',
 '   computing new velocity',
 'write lock file:',
 '',
 '   FemModel initialization elapsed time:   0.272385',
 '   Total Core solution elapsed time:       4.01669',
 '   Linear solver elapsed time:             2.36842 (59%)',
 '',
 '   Total elapsed time: 0 hrs 0 min 4 sec']

In [9]:
from export_netCDF import export_netCDF
export_netCDF(md, "inversion")

File inversion allready exist


Give a new name or "delete" to replace:  delete


WARNING type "int16" is unknown for "mask.ice_levelset"
WARNING type "int16" is unknown for "mask.ocean_levelset"
WARNING type "uint8" is unknown for "basalforcings.groundedice_melting_rate"
WARNING type "uint8" is unknown for "basalforcings.floatingice_melting_rate"
WARNING type "uint8" is unknown for "materials.rheology_n"
WARNING type "uint8" is unknown for "initialization.vz"
field md.solidearth.external is None
WARNING type "uint8" is unknown for "stressbalance.loadingforce"
qmu is skipped until it is more stable
